# Bonds prices calculation

In [1]:
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as web
from pandas_datareader.moex import MoexReader
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
bonds_payments = pd.read_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_payments.csv', index_col=0)
bonds_payments['Дата фактической выплаты'] = pd.to_datetime(bonds_payments['Дата фактической выплаты'])

bond_trade_codes = bonds_payments['Торговый код'].unique()

### Close prices and accrued interest import

In [3]:
moex_data = [MoexReader(code, start = '01.01.2016', end = '31.12.2016').read() for code in bond_trade_codes]
bonds_list_close = [moex_data[i][moex_data[i].BOARDID == 'TQOB'].drop_duplicates()['CLOSE'] for i in range(len(moex_data))]
bonds_list_accint = [moex_data[i][moex_data[i].BOARDID == 'TQOB'].drop_duplicates()['ACCINT'] for i in range(len(moex_data))]

bonds_list_close = pd.concat(bonds_list_close, axis = 1)
bonds_list_accint = pd.concat(bonds_list_accint, axis = 1)

bonds_list_close.columns = bond_trade_codes
bonds_list_accint.columns = bond_trade_codes

In [33]:
bonds_list_close.to_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_close_prices.csv')
bonds_list_accint.to_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_accint.csv')

### Principal calculation

In [7]:
dates = bonds_list_close.index

principal = pd.DataFrame(np.zeros([252, 20]), index = dates, columns = bonds_payments['Торговый код'].unique())

for i in range(len(dates)):
    payments_on_date = bonds_payments[bonds_payments['Дата фактической выплаты'] >= dates[i]]
    
    for ticker in bond_trade_codes:
        bond_principal = payments_on_date[payments_on_date['Торговый код'] == ticker]['Погашение номинала, RUB'].values.sum()
        
        principal[i:i+1][ticker] = bond_principal

/Users/andrey_lukyanov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [8]:
principal.to_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/principal.csv')

## Prices calculation

In [23]:
bonds_prices = np.zeros([252, 20])

for i in range(len(dates)):    
    for j in range(bonds_list_close.shape[1]):        
        bonds_prices[i, j] = bonds_list_close.iloc[i, j] * principal.iloc[i, j] / 100 + bonds_list_accint.iloc[i, j]
        
bonds_prices = pd.DataFrame(bonds_prices, index = dates, columns = bond_trade_codes)

In [32]:
bonds_prices.to_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_prices.csv')

## Loading existing data

In [6]:
bonds_prices = pd.read_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_prices.csv', index_col='TRADEDATE', parse_dates=True)

In [4]:
irr_matrix = pd.read_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_irrs.csv', index_col=0)

# IRR calcultation

In [34]:
def npv(irr, cfs, yrs):  
    return np.sum(cfs / (1. + irr) ** yrs)

In [35]:
from scipy.optimize import fsolve

def irr(cfs, yrs, x0):
    return np.asscalar(fsolve(npv, x0=x0, args=(cfs, yrs)))

In [36]:
def maturities_irrs(date_number):
    irrs = []
    maturities = []
    payments_on_date = bonds_payments[bonds_payments['Дата фактической выплаты'] >= dates[date_number]]
    
    for ticker in bonds_payments['Торговый код'].unique():
    
        payments = payments_on_date[payments_on_date['Торговый код'] == ticker]['Сумма купона, RUB'] + payments_on_date[payments_on_date['Торговый код'] == ticker]['Погашение номинала, RUB'].values
        principal = payments_on_date[payments_on_date['Торговый код'] == ticker]['Погашение номинала, RUB'].values.sum()
        payments_dates = (payments_on_date[payments_on_date['Торговый код'] == ticker]['Дата фактической выплаты'] - dates[date_number]).apply(lambda x: x.days)/365
    
        investment = bonds_prices.iloc[date_number][ticker]
        payments = np.insert(np.array(payments), 0, -investment)
        payments_dates = np.insert(np.array(payments_dates), 0, 0)
        
        if np.isnan(payments[0]) == True:
            irrs.append(np.nan)
            maturities.append(np.nan)
        else:
            bond_irr = irr(cfs=payments, yrs=payments_dates, x0=0.10)
            irrs.append(bond_irr)
            maturities.append(payments_dates[-1])
        
    return (np.array([maturities]), np.array([irrs]))   

In [37]:
irr_matrix = pd.DataFrame(np.zeros([252, 20]), index = dates, columns = bonds_payments['Торговый код'].unique())
maturity_matrix = pd.DataFrame(np.zeros([252, 20]), index = dates, columns = bonds_payments['Торговый код'].unique())

In [38]:
for i in range(len(dates)):
    maturity_matrix[dates[i]:dates[i]], irr_matrix[dates[i]:dates[i]] = maturities_irrs(i)    

/Users/andrey_lukyanov/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [39]:
maturity_matrix.to_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_maturities.csv')
irr_matrix.to_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_irrs.csv')

## BOOTSRAP

1. Сортируем бонды по их длине
2. Первую ставку взять из IRR наиболее короткого бонда
3. Далее решать каждый бонд при условии более коротких ставок.